In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import zipfile
import os

# Load from Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# zip_path = '/content/drive/MyDrive/Speech_Emotion_Recogntion/audio_src/combined_dataset.zip'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall('/content/')

Mounted at /content/drive


In [3]:
source_folder = '/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset'
destination_folder = '/content/audio_files'

In [6]:
!zip -r /content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset.zip "/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset"

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_429.wav (deflated 26%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_430.wav (deflated 26%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_431.wav (deflated 24%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_432.wav (deflated 25%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_433.wav (deflated 24%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_434.wav (deflated 21%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_435.wav (deflated 27%)
  adding: content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_436.wav (deflated 10%)
  adding: 

In [7]:
!cp "/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset.zip" /content/
!unzip /content/combined_dataset.zip -d /content/audio_files/

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_429.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_430.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_431.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_432.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_433.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_434.wav  
  inflating: /content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset/CREMA-D_disgust_435.wav  
  inflating: /content/audio_files/content/drive

# Test on 1 random file

In [ ]:
file_path = "/content/combined_dataset/JL_angry_58.wav"
y, sr = librosa.load(file_path, sr=None)  # Load with native sample rate

print("Sample rate : " + str(sr))

# Play original audio
print("Original audio :")
display(Audio(y, rate=sr))

# Show original sound wave
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y, sr=sr)
plt.title("Original sound wave")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

# Normalization based on maximum amplitude
y_normalized = y / np.max(np.abs(y))

# Remove silence from the beginning and end
y_trimmed, index = librosa.effects.trim(y_normalized, top_db=20)

# Resampling at 16000 Hz
target_sr = 16000
y_resampled = librosa.resample(y_trimmed, orig_sr=sr, target_sr=target_sr)

# Calculate target length (3 seconds)
target_length = target_sr * 3  # 3 seconds

# Calculate how much padding is needed at the start and end
current_length = len(y_resampled)
padding_length = target_length - current_length

if padding_length > 0:
    # Split padding equally between start and end
    pad_start = padding_length // 2
    pad_end = padding_length - pad_start

    # Add padding at the start and end
    y_padded = np.pad(y_resampled, (pad_start, pad_end), mode='constant')
else:
    # If audio is longer than 3 seconds, crop it
    y_padded = y_resampled[:target_length]

# Play audio after padding
print("Audio after padding :")
display(Audio(y_padded, rate=target_sr))

# Show padded sound wave
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y_padded, sr=target_sr)
plt.title("Padded sound wave")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

KeyboardInterrupt: 

In [ ]:
#test
import librosa
import numpy as np
from scipy.signal import butter, lfilter
import random
import pyrubberband as pyrb
import matplotlib.pyplot as plt

def preprocess_audio(file_path, target_sr=16000, target_duration=3):
    """
    Prétraitement original comme dans votre code
    """
    # Load with native sample rate
    y, sr = librosa.load(file_path, sr=None)

    # Normalization based on maximum amplitude
    y_normalized = y / np.max(np.abs(y))

    # Remove silence from the beginning and end
    y_trimmed, index = librosa.effects.trim(y_normalized, top_db=20)

    # Resampling at 16000 Hz
    y_resampled = librosa.resample(y_trimmed, orig_sr=sr, target_sr=target_sr)

    # Calculate target length (3 seconds)
    target_length = target_sr * target_duration

    # Calculate padding needed
    current_length = len(y_resampled)
    padding_length = target_length - current_length

    if padding_length > 0:
        # Split padding equally between start and end
        pad_start = padding_length // 2
        pad_end = padding_length - pad_start
        # Add padding at the start and end
        y_padded = np.pad(y_resampled, (pad_start, pad_end), mode='constant')
    else:
        # If audio is longer than 3 seconds, crop it
        y_padded = y_resampled[:target_length]

    return y_padded, target_sr

def augment_audio(audio, sr):
    """
    Fonctions d'augmentation appliquées sur l'audio prétraité
    """
    augmented_samples = [audio]  # Original audio

    # 1. Ajout de bruit blanc
    def add_white_noise(y, noise_factor=0.005):
        noise = np.random.normal(0, 1, len(y))
        return y + noise_factor * noise

    # 2. Changement de vitesse
    def change_speed(y, speed_factor):
        return pyrb.time_stretch(y, sr, speed_factor)

    # 3. Changement de pitch
    def change_pitch(y, pitch_factor):
        return pyrb.pitch_shift(y, sr, pitch_factor)

    # 4. Filtre passe-bas
    def apply_lowpass(y, cutoff=1000):
        nyquist = sr // 2
        normal_cutoff = cutoff / nyquist
        b, a = butter(4, normal_cutoff, btype='low')
        return lfilter(b, a, y)

    # 5. Time shifting
    def time_shift(y, shift_max=0.3):
        shift = int(random.random() * shift_max * len(y))
        return np.roll(y, shift)

    # 6. Time masking
    def time_mask(y, mask_size=0.1):
        mask_length = int(len(y) * mask_size)
        mask_start = random.randint(0, len(y) - mask_length)
        y_masked = y.copy()
        y_masked[mask_start:mask_start + mask_length] = 0
        return y_masked

    # Appliquer les augmentations
    augmented_samples.extend([
        add_white_noise(audio),
        change_speed(audio, 0.9),
        change_speed(audio, 1.1),
        change_pitch(audio, -2),
        change_pitch(audio, 2),
        apply_lowpass(audio),
        time_shift(audio),
        time_mask(audio)
    ])

    return augmented_samples

# Exemple d'utilisation
if __name__ == "__main__":
    file_path = "/content/combined_dataset/JL_angry_58.wav"

    # Prétraitement original
    processed_audio, sr = preprocess_audio(file_path)

    # Visualisation audio original prétraité
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(processed_audio, sr=sr)
    plt.title("Preprocessed audio")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()
    display(Audio(processed_audio, rate=sr))

    # Application des augmentations
    augmented_samples = augment_audio(processed_audio, sr)

    # Visualisation des versions augmentées
    for i, audio in enumerate(augmented_samples):
        plt.figure(figsize=(10, 4))
        librosa.display.waveshow(audio, sr=sr)
        plt.title(f"Augmented audio {i}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.show()
        display(Audio(audio, rate=sr))

In [9]:
def preprocess_audio(file_path, target_duration=3, target_sr=16000):
    """
    Charge un fichier audio, enlève les silences, le rééchantillonne,
    et ajoute du padding pour que la durée soit égale à target_duration (en secondes).

    Args:
    - file_path (str): Le chemin du fichier audio.
    - target_duration (float): La durée cible de l'audio en secondes.
    - target_sr (int): Le taux d'échantillonnage cible pour l'audio.

    Returns:
    - y_padded (numpy.ndarray): L'audio traité avec padding ajouté si nécessaire.
    """
    # Charger l'audio
    y, sr = librosa.load(file_path, sr=None)  # Charger avec le taux d'échantillonnage natif

    # Normalisation basée sur l'amplitude maximale
    y_normalized = y / np.max(np.abs(y))

    # Suppression des silences au début et à la fin
    y_trimmed, index = librosa.effects.trim(y_normalized, top_db=20)

    # Rééchantillonnage à target_sr
    y_resampled = librosa.resample(y_trimmed, orig_sr=sr, target_sr=target_sr)

    # Calcul de la durée cible en nombre d'échantillons
    target_length = target_sr * target_duration  # Durée cible en nombre d'échantillons
    current_length = len(y_resampled)
    padding_length = target_length - current_length

    if padding_length > 0:
        # Répartition du padding entre le début et la fin
        pad_start = padding_length // 2
        pad_end = padding_length - pad_start

        # Ajouter du padding au début et à la fin
        y_padded = np.pad(y_resampled, (pad_start, pad_end), mode='constant')
    else:
        # Si l'audio est plus long que la durée cible, on le coupe
        y_padded = y_resampled[:target_length]

    return y_padded, target_sr



def generate_spectrogram(y, sr, output_file):
    """
    Génère un spectrogramme Mel sans bords noirs ni compression
    et l'enregistre en PNG.
    """
    # Calculer le spectrogramme Mel
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_db = librosa.power_to_db(S, ref=np.max)

    # Déterminer la taille optimale pour le spectrogramme
    plt.figure(figsize=(S_db.shape[1] / 100, S_db.shape[0] / 100), dpi=100)
    plt.axis('off')  # Supprimer les axes
    librosa.display.specshow(S_db, sr=sr, fmax=8000, x_axis=None, y_axis=None)

    # Sauvegarder le spectrogramme avec un cadre parfaitement ajusté
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Éliminer les marges
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0, dpi=100, transparent=True)
    plt.close()

In [10]:
# Dossier contenant les fichiers du dataset combiné
input_dir = "/content/audio_files/content/drive/MyDrive/Speech_Emotion_Recogntion/combined_dataset"  # Modifier ce chemin pour le répertoire combined_dataset
output_dir = "/content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset"  # Dossier où les spectrogrammes seront enregistrés

# Créer le dossier de sortie s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Parcourir les fichiers dans le dossier combined_dataset
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.wav'):
            # Obtenir le chemin du fichier audio
            file_path = os.path.join(root, file)

            try:
                # Appeler la fonction preprocess_audio pour prétraiter l'audio
                y_padded, sr = preprocess_audio(file_path)

                # Générer le chemin du fichier de sortie pour le spectrogramme
                output_file = os.path.join(output_dir, file.replace('.wav', '.png'))

                # Appeler la fonction generate_spectrogram pour créer et sauvegarder le spectrogramme
                generate_spectrogram(y_padded, sr, output_file)

                print(f"Spectrogramme généré et sauvegardé : {output_file}")

            except Exception as e:
                print(f"Erreur avec le fichier {file_path}: {e}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/TESS_fearful_6.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/SAVEE_angry_74.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/ESD_surprise_410.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/CREMA-D_disgust_488.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/RAVDESS_neutral_25.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recogntion/audio_representation_big_dataset/ESD_neutral_53.png
Spectrogramme généré et sauvegardé : /content/drive/MyDrive/Speech_Emotion_Recognti

In [14]:
# Vérification si le dossier existe
if os.path.exists(output_dir):
    # Compte le nombre d'éléments dans le dossier
    num_elements = len(os.listdir(output_dir))
    print(f"Le dossier contient {num_elements} éléments.")
else:
    print(f"Le dossier {input_dir} n'existe pas. Vérifiez le chemin.")

Le dossier contient 5501 éléments.
